In [1]:
import torch
from collections import OrderedDict

In [7]:
from dataset.dataset import MultiModalDataset
from mmcv_model.mmcv_csn import ResNet3dCSN
from mmcv_model.scheduler import GradualWarmupScheduler

from model.average_model import MultiModalModel
from model.multimodal_neck import MultiModalNeck
from model.simple_head import SimpleHead
from model.flow_autoencoder import FlowAutoencoder

/home/myuser1/miniconda3/envs/wlasl/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [3]:
ls

 auto.sh                  __pycache__/
 dataset/                 rgb_backbone.pth
 dataset_output.ipynb     rgb_only.py
 depthepoch_100.pth       right_handepoch_100.pth
 depth_only.py            right_hand_only.py
 faceepoch_100.pth        setup.sh
 face_only.py             seven-sees-net-mid-fusion.ipynb
 flow_backbone.pth        skeletonepoch_100.pth
 flow_only.py             skeleton_only.py
 flow_output.ipynb        train_fusion.py
'Freezing Layers.ipynb'   train_fusion_v1.py
 left_handepoch_100.pth   train_rgb_autsl.py
 left_hand_only.py        train_rgb.py
 mid_fusion.py           'Trim Checkpoint.ipynb'
 mmcv_model/              wlasl2000.py
 model/


In [11]:
checkpoint = torch.load('./flow_encoder_flow_wlasl100_100e.pth')

In [12]:
checkpoint.keys()

odict_keys(['flow_backbone.conv1.conv.weight', 'flow_backbone.conv1.bn.weight', 'flow_backbone.conv1.bn.bias', 'flow_backbone.conv1.bn.running_mean', 'flow_backbone.conv1.bn.running_var', 'flow_backbone.conv1.bn.num_batches_tracked', 'flow_backbone.layer1.0.conv1.conv.weight', 'flow_backbone.layer1.0.conv1.bn.weight', 'flow_backbone.layer1.0.conv1.bn.bias', 'flow_backbone.layer1.0.conv1.bn.running_mean', 'flow_backbone.layer1.0.conv1.bn.running_var', 'flow_backbone.layer1.0.conv1.bn.num_batches_tracked', 'flow_backbone.layer1.0.conv2.0.conv.weight', 'flow_backbone.layer1.0.conv2.0.bn.weight', 'flow_backbone.layer1.0.conv2.0.bn.bias', 'flow_backbone.layer1.0.conv2.0.bn.running_mean', 'flow_backbone.layer1.0.conv2.0.bn.running_var', 'flow_backbone.layer1.0.conv2.0.bn.num_batches_tracked', 'flow_backbone.layer1.0.conv3.conv.weight', 'flow_backbone.layer1.0.conv3.bn.weight', 'flow_backbone.layer1.0.conv3.bn.bias', 'flow_backbone.layer1.0.conv3.bn.running_mean', 'flow_backbone.layer1.0.conv

In [13]:
keys = list(checkpoint.keys())

In [14]:
for key in keys:
    print(key)

flow_backbone.conv1.conv.weight
flow_backbone.conv1.bn.weight
flow_backbone.conv1.bn.bias
flow_backbone.conv1.bn.running_mean
flow_backbone.conv1.bn.running_var
flow_backbone.conv1.bn.num_batches_tracked
flow_backbone.layer1.0.conv1.conv.weight
flow_backbone.layer1.0.conv1.bn.weight
flow_backbone.layer1.0.conv1.bn.bias
flow_backbone.layer1.0.conv1.bn.running_mean
flow_backbone.layer1.0.conv1.bn.running_var
flow_backbone.layer1.0.conv1.bn.num_batches_tracked
flow_backbone.layer1.0.conv2.0.conv.weight
flow_backbone.layer1.0.conv2.0.bn.weight
flow_backbone.layer1.0.conv2.0.bn.bias
flow_backbone.layer1.0.conv2.0.bn.running_mean
flow_backbone.layer1.0.conv2.0.bn.running_var
flow_backbone.layer1.0.conv2.0.bn.num_batches_tracked
flow_backbone.layer1.0.conv3.conv.weight
flow_backbone.layer1.0.conv3.bn.weight
flow_backbone.layer1.0.conv3.bn.bias
flow_backbone.layer1.0.conv3.bn.running_mean
flow_backbone.layer1.0.conv3.bn.running_var
flow_backbone.layer1.0.conv3.bn.num_batches_tracked
flow_backb

In [17]:
# Custom multimodal model
flow_backbone = ResNet3dCSN(
    pretrained2d=False,
    pretrained=None,
    depth=50,
    with_pool2=False,
    bottleneck_mode='ir',
    norm_eval=True,
    zero_init_residual=False,
    bn_frozen=True
)

neck = MultiModalNeck()
head = SimpleHead(num_classes=400,
                  in_channels=2048,
                  dropout_ratio=0.5,
                  init_std=0.01)


flow_stream = FlowAutoencoder(flow_backbone=flow_backbone,
                        neck=neck,
                        head=head)

flow_stream.load_state_dict(checkpoint)

<All keys matched successfully>

In [54]:
for key in keys:
    if key[:4]=='head':
        del checkpoint[key]

In [55]:
keys = list(checkpoint.keys())
for key in keys:
    print(key)

rgb_backbone.conv1.conv.weight
rgb_backbone.conv1.bn.weight
rgb_backbone.conv1.bn.bias
rgb_backbone.conv1.bn.running_mean
rgb_backbone.conv1.bn.running_var
rgb_backbone.conv1.bn.num_batches_tracked
rgb_backbone.layer1.0.conv1.conv.weight
rgb_backbone.layer1.0.conv1.bn.weight
rgb_backbone.layer1.0.conv1.bn.bias
rgb_backbone.layer1.0.conv1.bn.running_mean
rgb_backbone.layer1.0.conv1.bn.running_var
rgb_backbone.layer1.0.conv1.bn.num_batches_tracked
rgb_backbone.layer1.0.conv2.0.conv.weight
rgb_backbone.layer1.0.conv2.0.bn.weight
rgb_backbone.layer1.0.conv2.0.bn.bias
rgb_backbone.layer1.0.conv2.0.bn.running_mean
rgb_backbone.layer1.0.conv2.0.bn.running_var
rgb_backbone.layer1.0.conv2.0.bn.num_batches_tracked
rgb_backbone.layer1.0.conv3.conv.weight
rgb_backbone.layer1.0.conv3.bn.weight
rgb_backbone.layer1.0.conv3.bn.bias
rgb_backbone.layer1.0.conv3.bn.running_mean
rgb_backbone.layer1.0.conv3.bn.running_var
rgb_backbone.layer1.0.conv3.bn.num_batches_tracked
rgb_backbone.layer1.0.downsample.c

In [56]:
x ='rgb_backbone.layer4.2.conv3.bn.weight'

In [57]:
x[13:]

'layer4.2.conv3.bn.weight'

In [58]:
checkpoint=OrderedDict((k[13:] if k[:13] == 'rgb_backbone.' else k, v) for k, v in checkpoint.items())

In [59]:
keys = list(checkpoint.keys())
for key in keys:
    print(key)

conv1.conv.weight
conv1.bn.weight
conv1.bn.bias
conv1.bn.running_mean
conv1.bn.running_var
conv1.bn.num_batches_tracked
layer1.0.conv1.conv.weight
layer1.0.conv1.bn.weight
layer1.0.conv1.bn.bias
layer1.0.conv1.bn.running_mean
layer1.0.conv1.bn.running_var
layer1.0.conv1.bn.num_batches_tracked
layer1.0.conv2.0.conv.weight
layer1.0.conv2.0.bn.weight
layer1.0.conv2.0.bn.bias
layer1.0.conv2.0.bn.running_mean
layer1.0.conv2.0.bn.running_var
layer1.0.conv2.0.bn.num_batches_tracked
layer1.0.conv3.conv.weight
layer1.0.conv3.bn.weight
layer1.0.conv3.bn.bias
layer1.0.conv3.bn.running_mean
layer1.0.conv3.bn.running_var
layer1.0.conv3.bn.num_batches_tracked
layer1.0.downsample.conv.weight
layer1.0.downsample.bn.weight
layer1.0.downsample.bn.bias
layer1.0.downsample.bn.running_mean
layer1.0.downsample.bn.running_var
layer1.0.downsample.bn.num_batches_tracked
layer1.1.conv1.conv.weight
layer1.1.conv1.bn.weight
layer1.1.conv1.bn.bias
layer1.1.conv1.bn.running_mean
layer1.1.conv1.bn.running_var
layer1

In [60]:
torch.save(checkpoint, 'right_hand_backbone.pth')